In [7]:
import ansys.fluent.core as pyfluent
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define functions

In [8]:
def change_zone_type(zone_name, zone_type):
    solver.tui.define.boundary_conditions.modify_zones.zone_type(
        zone_name,
        zone_type
    )
    
def make_periodic(zone1, zone2):
    solver.tui.mesh.modify_zones.create_periodic_interface('conformal',
                                                           zone1,
                                                           zone1,
                                                           zone2,
                                                           'no',
                                                           'yes','','','','')
    
def set_pressure(pressure):
    solver.tui.define.periodic_conditions.pressure_gradient_specification(pressure)
    
def report(name, type, field = None , report_type = None, zone_names = None, definition = None):
    if type == "surface":
        if field == None:
            solver.solution.report_definitions.surface[name] = {"report_type" : report_type , "surface_names" : zone_names}
        else:
            solver.solution.report_definitions.surface[name] = {"field" : field ,"report_type" : report_type , "surface_names" : zone_names}
    elif type == "volume":
        if field == None:
            solver.solution.report_definitions.volume[name] = {"report_type" : report_type, "zone_names" : zone_names}
        else:
            solver.solution.report_definitions.volume[name] = {"field" : field, "report_type" : report_type, "zone_names" : zone_names}
    elif type == "sve":
        solver.solution.report_definitions.single_val_expression[name] = {"define" : definition}
    
def report_output(name):
    filename = f"report_{np.random.randint(1000000)}.out"
    solver.tui.define.parameters.output_parameters.create('report-definition', name)
    solver.tui.define.parameters.output_parameters.write_all_to_file(filename)
        
    with open(filename, 'r') as file:
        file_content = file.read()

    data_lines = file_content.split('\n')
    data = []
    for line in data_lines:
        if line.strip() != "":
            columns = line.split()
            if len(columns) == 3:
                data.append(columns)
                
    report = float(data[0][1])

    solver.tui.define.parameters.output_parameters.delete(f'{name}-op','yes') 

    os.remove(filename)

    return report
    
def create_report_plot(name, report_definitions, print=False):
    solver.solution.monitor.report_plots[name] = {"report_defs" : report_definitions,"print" : print}
    
def initialize_calculate(iterations):
    solver.solution.initialization.standard_initialize()
    solver.solution.run_calculation.iterate(iter_count = iterations)

def create_contour(name, field, surfaces_list):
    solver.results.graphics.contour[name] = {"field" : field, "surfaces_list" : surfaces_list}
    
def duplicate_material(name, source_material):
    solver.tui.define.materials.change_create(source_material,name,'no','no','no','no','no','no','no','no')
    
def interpret_udf(udf_path):
    solver.tui.define.user_defined.interpreted_functions(udf_path, '', '', '')

def user_defined_mixture(material1, material2, material3, user_specified_diffusivity, energy=False):
    solver.tui.define.models.species.species_transport('yes', 'mixture-template')
    solver.tui.define.models.species.diffusion_energy_source('no')
    solver.tui.define.materials.change_create('mixture-template',
                                    'mixture',
                                    'yes',
                                    '3',
                                    material1,
                                    material2,
                                    material3,
                                    '0',
                                    '0',
                                    'yes',
                                    'volume-weighted-mixing-law',
                                    'yes',
                                    'mixing-law',
                                    'yes',
                                    'mass-weighted-mixing-law',
                                    'yes',
                                    'mass-weighted-mixing-law',
                                    'yes',
                                    'user-defined',
                                    user_specified_diffusivity,                                   
                                    'no',
                                    'yes')
    if energy==False:
        solver.setup.models.energy = {"enabled" : False} # turn off energy, which is automatically turned on when creating mixture-template

    # Delete materials created from default mixture-template to keep things clean
    solver.tui.define.materials.delete('nitrogen')
    solver.tui.define.materials.delete('oxygen')
    solver.tui.define.materials.delete('water-vapor')

def read_expressions(path):
    solver.tui.define.named_expressions.import_from_tsv(path)
    
def save_differential():
    solver.tui.define.models.species.save_gradients('yes')
    solver.solution.run_calculation.iterate(iter_count = 1)
    
def adjust_source_terms(dye = "species-1"):
    solver.setup.cell_zone_conditions.fluid['fluid_zone'].source_terms = {dye : ["uref"]} #maybe have to do full thing
    solver.setup.cell_zone_conditions.fluid['stationary_zone'].source_terms = {dye : ["urefs"]}
    
def boundary_conditions_halo(inner_wall):
    solver.setup.boundary_conditions.wall[inner_wall].species_spec = {"dm_kds" : "Specified Mass Fraction", "dye" : "Specified Flux (Mass)"}
    
def dummy_field_function():
    solver.tui.define.custom_field_functions.define('"dm_kds_patch"','diffl_dye')

def change_expression(expression, new_value):
    solver.setup.named_expressions[expression] = {"definition" : str(new_value)}
    
def update_udf(path, line, new_line):
    values = line, new_line

    file_path = path
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if line < len(lines):
            lines[line] = new_line

    with open(file_path, 'w') as file:
        file.writelines(lines)
        
def patch_dm_kds(dm_kds = "species-0"):
    solver.tui.solve.patch('fluid_zone','stationary_zone',(),'species-0','yes','dm_kds_patch')
    
def patch_dye(dye = "species-1"):
    solver.tui.solve.patch('fluid_zone','stationary_zone',(),'species-1','no','0')
    
def pe_to_dmol(vx_avg, L, peclet_numbers):
    dmols = (float(vx_avg)*L)/peclet_numbers
    return dmols

# Define inputs

In [9]:
project_names = ['12_spikes_40', '12_spikes_50', '12_spikes_50_coreshell', '12_spikes_60', '14_spikes_40']

In [10]:
show_gui = True

k_list = [1, 2, 4, 8, 16]

peclet_numbers = np.array([0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128])

L = "domain"

model = "laminar"

velocity_iterations = 500

dax_iterations = 500

pressure = -1e+08

coeff_ratio = 0.3

processor_count = 4

µ = 0.001003

skip_velocity_field = False

In [14]:
thesis_folder = "C:/Users/yassi/OneDrive/Documents/School/Thesis"

udf_path = f"{thesis_folder}/UDFs/diffusion_coefficients.txt"
expressions_path = f"{thesis_folder}/UDFs/dax_expressions.tsv"

solver = pyfluent.launch_fluent(precision="double", processor_count=processor_count, mode="solver", show_gui=show_gui, start_transcript=False)

# Loop

In [15]:
for project_name in project_names:
    print(f"Now starting with {project_name}")
    directory = f"{thesis_folder}/Projects - Copy/{project_name}"

    results_directory = os.path.join(directory,'results')
    case_data_directory = os.path.join(directory,'case_data_files')

    if not os.path.exists(results_directory):
        os.mkdir(results_directory)
                
    if not os.path.exists(case_data_directory):
        os.mkdir(case_data_directory)
        
    if skip_velocity_field == True:
        solver.file.read_case_data(file_name=os.path.join(case_data_directory,f"{project_name}-velocity-field.cas.h5"))
    else:    
        solver.file.read_mesh(file_name=os.path.join(directory,f"{project_name}.msh"))
    
        change_zone_type("interface", "wall")
        change_zone_type("inlet-stationary_zone", "wall")
        change_zone_type("outlet-stationary_zone", "wall")

        solver.setup.models.viscous.model = model

        make_periodic("inlet-fluid_zone", "outlet-fluid_zone")

        solver.tui.define.materials.copy('fluid','water-liquid')
        solver.setup.cell_zone_conditions.fluid['fluid_zone'] = {"material" : "water-liquid", "sources" : True}

        solver.tui.define.boundary_conditions.zone_type('stationary_zone','fluid')
        solver.setup.cell_zone_conditions.fluid['stationary_zone'] = {"material" : "water-liquid", "sources" : True}
        solver.tui.define.materials.delete('air')

        set_pressure(pressure)
        
        report(name = "vx_avg", type = "volume", field = "x-velocity", report_type = "volume-average", zone_names = ["fluid_zone"])

        create_report_plot("vx_avg-plot", ["vx_avg"], True)

        solver.tui.solve.monitors.residual.check_convergence('no','no','no','no')

        initialize_calculate(iterations = velocity_iterations)

        solver.file.write(file_type = "case-data", file_name = f'{case_data_directory}/{project_name}-velocity-field')
        
    vx_avg = report_output("vx_avg")
    permeability = (vx_avg * µ) / (-pressure)
    d_eff = np.sqrt(32*permeability)
    
    duplicate_material("dye", "water-liquid")
    
    duplicate_material("dm_kds", "water-liquid")

    interpret_udf(udf_path)

    user_defined_mixture("dm_kds", "dye", "water-liquid", "user_specified_diff") # water needs to be last

    read_expressions(expressions_path)
    
    report(name = "db_dx_s", type = "volume", report_type = "volume-average", field = "expr:dB_dx", zone_names = ["stationary_zone"])
    report(name = "db_dx_m", type = "volume", report_type = "volume-average", field = "expr:dB_dx", zone_names = ["fluid_zone"])
    report(name = "dax_1_s", type = "volume", report_type = "volume-average", field = "expr:Dax1", zone_names = ["stationary_zone"])
    report(name = "dax_1_m", type = "volume", report_type = "volume-average", field = "expr:Dax1", zone_names = ["fluid_zone"])
    report(name = "dax", type = "sve", definition = "Dax")
    
    create_report_plot("dax_plot", ["dax"], print=True)

    solver.tui.solve.monitors.residual.check_convergence('no','no','no','no','no','no')

    save_differential()

    change_zone_type("interface", "interior")
    make_periodic("inlet-stationary_zone", "outlet-stationary_zone")
    
    adjust_source_terms(dye="species-1")

    if "coreshell" in project_name:
        boundary_conditions_halo("inner_wall")
        
    solver.solution.controls.equations = {"flow" : False}
    solver.solution.controls.equations = {"species-0" : False}

    dummy_field_function()
    
    # Compute some mesh parameters that are used later of useful to know
    
    report(name = "fluid_vol", type = "volume", report_type = "volume-zonevol", zone_names = ["fluid_zone"])
    fluid_vol = report_output("fluid_vol")
    
    report(name = "stationary_vol", type = "volume", report_type = "volume-zonevol", zone_names = ["stationary_zone"])
    stationary_vol = report_output("stationary_vol")

    report(name = "d_dom", type = "surface", report_type="surface-vertexmax", field = "x-coordinate", zone_names = ["outlet-fluid_zone"])
    d_dom = report_output("d_dom")

    report(name = "interface_surf", type = "surface", report_type="surface-area", zone_names = ["interface"])
    interface_surface = report_output("interface_surf")
    
    unit_cell_reduction = (d_dom**3)/(fluid_vol+stationary_vol)
    
    if L == "effective":
        diffusion_coefficients = pe_to_dmol(vx_avg = vx_avg, L = d_eff, peclet_numbers = peclet_numbers)
    elif L == "domain":
        diffusion_coefficients = pe_to_dmol(vx_avg = vx_avg, L = d_dom, peclet_numbers = peclet_numbers)
    
    df = pd.DataFrame(columns = ['project_name', 'fluid_vol', 'stationary_vol', 'interface_surface', 'd_dom', 'vx_avg', 'pressure', 'permeability', 'd_eff', 'k', 'dmol', 'peclet', 'dax', 'unit_cell_reduction', 'L'])  

    for k in k_list:

        K = k*(fluid_vol/stationary_vol)

        change_expression("k", str(k))
        change_expression("K", str(K))

        update_udf(udf_path, 11, f"real K = {K};\n")

        for i, dmol in enumerate(diffusion_coefficients):
            change_expression("Dmol", str(dmol))
            change_expression("Dpor", str(dmol*coeff_ratio))
            update_udf(udf_path, 12, f"real D_mol = {dmol};\n")
            update_udf(udf_path, 13, f"real D_por = {K*dmol*coeff_ratio};\n")
            
            interpret_udf(udf_path)
            
            # ORDER OF PATCHING IS IMPORTANT FOR SOME REASON (OTHER ORDER GIVES WRONG RESULT AFTER FIRST RUN)
            patch_dye()
            patch_dm_kds()
            
            solver.solution.run_calculation.iterate(iter_count = dax_iterations)
            
            dax = report_output("dax")/100
                        
            new_row = [project_name, fluid_vol, stationary_vol, interface_surface, d_dom, vx_avg, pressure, permeability, d_eff, k, dmol, peclet_numbers[i], dax, unit_cell_reduction, L] 
            
            df.loc[len(df)] = new_row
            display(df.iloc[-1:])

    solver.tui.file.write_case_data(f'{case_data_directory}/{project_name}_b-field')

    df.to_csv(f'{results_directory}/results_{project_name}.csv')

Now starting with 12_spikes_40


RuntimeError: CAR: invalid argument [1]: wrong type [not a pair]
Error Object: #f

In [ ]:
solver.exit()